<a href="https://colab.research.google.com/github/dusarp/dance-bits-experiments/blob/main/mel_spectrogram_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import numpy as np

In [ ]:
#mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving real-studio-vid1.npy to real-studio-vid1.npy


In [ ]:
# Load the video file
file_path = "/content/real-studio-vid1.npy"
data = np.load(file_path)

In [ ]:
# Print the data or check its shape, etc.
print(data)
print(data.shape)

[[-24.857124  -15.146767  -12.073616  ... -13.662367  -11.894629
  -19.208868 ]
 [-21.532417  -12.769215   -8.280472  ...  -7.190918   -5.780098
  -10.4868355]
 [-20.105932  -15.604622  -15.66144   ... -15.08651   -13.744595
  -13.981142 ]
 ...
 [-36.756767  -36.76602   -38.58355   ... -39.628185  -41.021534
  -45.92129  ]
 [-44.233425  -40.104527  -42.53934   ... -43.948376  -45.736164
  -50.73719  ]
 [-53.981346  -50.331394  -54.661453  ... -54.401543  -56.282005
  -58.12305  ]]
(128, 2801)


In [ ]:
# Step 1: Normalize the spectrogram to the range [0, 1]
mel_spec_min = data.min()
mel_spec_max = data.max()
mel_spec_normalized = (data - mel_spec_min) / (mel_spec_max - mel_spec_min)

# Step 2: Scale the normalized values to the range [-0.5, 0.5]
mel_spec_scaled = mel_spec_normalized - 0.5

In [ ]:
print(mel_spec_scaled)
print(mel_spec_scaled.shape)

[[ 0.18928593  0.31066543  0.3490798  ...  0.3292204   0.3513171
   0.25988913]
 [ 0.2308448   0.34038484  0.3964941  ...  0.4101135   0.4277488
   0.3689146 ]
 [ 0.24867582  0.30494225  0.304232   ...  0.31141865  0.32819253
   0.32523572]
 ...
 [ 0.0405404   0.0404247   0.01770562 ...  0.00464767 -0.01276916
  -0.07401615]
 [-0.05291781 -0.00130659 -0.03174177 ... -0.0493547  -0.07170206
  -0.13421488]
 [-0.17476684 -0.12914243 -0.18326816 ... -0.18001929 -0.20352507
  -0.22653812]]
(128, 2801)


In [ ]:
#unmoount Google Drive
#drive.flush_and_unmount()

In [ ]:
class MelSpectrogramCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(MelSpectrogramCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 32 * 32)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x